<a href="https://colab.research.google.com/github/jcs-lambda/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Assignment

## prep

In [0]:
# impute the median latitude, longitude, and gps_height from other geographical features
# working through the features from most to least specific until there are no nan's or 
# no more geographical features to check
# must set impute_source to the source dataframe before calling
# this function is meant to be used as the target of a DataFrame.apply(impute_gps, axis=1) call
#   to be used on a subset of the dataframe where latitude and longitude are nan's
# parameter 'row' = a pandas series. a single row from a dataframe
impute_source = None

def impute_gps(row):
  assert impute_source is not None, 'impute_source not set'
  geo_features = ['basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward']
  geo_features_most_to_least_specific = impute_source[geo_features].nunique().sort_values(ascending=False).index.tolist()
  for geo in geo_features_most_to_least_specific:
    if row[geo] == '':
      continue
    # only calculate if it is nan
    if np.isnan(row['latitude']):
      row['latitude'] = impute_source[impute_source[geo] == row[geo]]['latitude'].median()
    if np.isnan(row['longitude']):
      row['longitude'] = impute_source[impute_source[geo] == row[geo]]['longitude'].median()
    if np.isnan(row['gps_height']):
      row['gps_height'] = impute_source[impute_source[geo] == row[geo]]['gps_height'].median()
    # if none of the values are nans, stop looking
    if not (np.isnan(row['latitude']) and np.isnan(row['longitude']) and np.isnan(row['gps_height'])):
      break
  
  # return only what we changed
  return row[['gps_height', 'longitude', 'latitude']]

### **wrangle function**

In [0]:
def wrangle(dataframe):
  df = dataframe.copy()

  # high cardinality categoricals cleanup
  # tag missing values
  # fill nan's with empty strings
  # convert to lower case and remove all non-alphanumeric characters to group
  #   things that look alike into 1 thing
  hcc_to_clean = ['funder', 'installer', 'wpt_name', 'subvillage', 'ward', 'scheme_name']
  for column in hcc_to_clean:
    df[column + '_MISSING'] = (df[column].isnull()) | (df[column] == '') | (df[column].fillna('').str.match('^[^A-Za-z0-9]+$'))
    df[column] = df[column].fillna('').str.lower().str.replace('[^a-z0-9]', '')
  
  # also combine items in those same features more explicitly
  df['funder'] = df['funder'].str.replace('^quickwin$', 'quickwins')
  df['installer'] = df['installer'].str.replace('^notknown', '')
  df['wpt_name'] = df['wpt_name'].str.replace('^rc$', 'rcchurch')

  # boolean feature cleanup
  # tag missing values
  # convert to floats to keep the nan's for imputing
  boolean_features = ['public_meeting', 'permit']
  for column in boolean_features:
    df[column + '_MISSING'] = df[column].isnull()
    df[column] = df[column].astype(float)

  # impute missing latitude, longitude, gps_height
  # missing is defined as longitude = 0, latitude = -2e-08
  # gps_height is assumed missing for observations where the above are true
  #   (exploration has shown that gps_height = 0 for all observations that meet those conditions)
  # first, we convert these values to nans, then impute the values for each feature
  # the imputed values are the median of that feature for all observations in the dataframe that
  #   match that observation's other geographical information
  df['longitude'] = df['longitude'].replace(0, np.nan)
  df['latitude'] = df['latitude'].replace(-2e-08, np.nan)
  df.loc[df['longitude'].isnull(), 'gps_height'] = np.nan
  df['lat_long_MISSING'] = (df['longitude'].isnull()) | (df['latitude'].isnull())
  imputed_gps = df[df['lat_long_MISSING']].apply(impute_gps, axis=1)
  df.update(imputed_gps, overwrite=False)     # overwrite=False means it will only overwrite nan values
  assert not df['longitude'].isnull().any(), 'null longitude'
  assert not df['latitude'].isnull().any(), 'null latitude'

  # When columns have zeros and shouldn't, they are like null values.
  # So we will replace the zeros with nulls, and impute missing values later.
  # Also create a "missing indicator" column, because the fact that
  # values are missing may be a predictive signal.
  cols_with_zeros = ['construction_year', 'gps_height', 'population']
  for col in cols_with_zeros:
      df[col] = df[col].replace(0, np.nan)
      df[col+'_MISSING'] = df[col].isnull()

  # Drop duplicate columns
  # including some features that are close duplicates - re-encoding of the same values
  duplicates = ['quantity_group', 'payment_type', 'waterpoint_type_group']
  df = df.drop(columns=duplicates)

  # Drop recorded_by (never varies) and id (always varies, random)
  unusable_variance = ['recorded_by', 'id']
  df = df.drop(columns=unusable_variance)

  # Convert date_recorded to datetime
  df['date_recorded'] = pd.to_datetime(df['date_recorded'], infer_datetime_format=True)

  # Extract components from date_recorded, then drop the original column
  df['year_recorded'] = df['date_recorded'].dt.year
  df['month_recorded'] = df['date_recorded'].dt.month
  df['day_recorded'] = df['date_recorded'].dt.day
  df = df.drop(columns='date_recorded')

  # Engineer feature: how many years from construction_year to date_recorded
  df['years'] = df['year_recorded'] - df['construction_year']
  df['years_MISSING'] = df['years'].isnull()

  return df

In [0]:
impute_source = train
df_train = wrangle(train)

impute_source = test
df_test = wrangle(test)

### assign X features matrix and y target vector

In [0]:
target = 'status_group'

In [19]:
cardinality = df_train.drop(columns=target).select_dtypes(exclude=['number', 'bool']).nunique()
low_cardinality = cardinality[cardinality<=50].index.tolist()
df_train[low_cardinality].dtypes

basin                    object
region                   object
scheme_management        object
extraction_type          object
extraction_type_group    object
extraction_type_class    object
management               object
management_group         object
payment                  object
water_quality            object
quality_group            object
quantity                 object
source                   object
source_type              object
source_class             object
waterpoint_type          object
dtype: object

In [0]:
features = df_train.columns.drop(target)

X_train = df_train[features]
X_test = df_test[features]

y_train = df_train[target]

## Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV

transformations = make_pipeline(
    ce.OneHotEncoder(cols=low_cardinality, use_cat_names=True),
    ce.OrdinalEncoder(),
)

pipeline = make_pipeline(
    SimpleImputer(),
    RandomForestClassifier(random_state=13, n_jobs=-1)
)

param_distributions = {
    'simpleimputer__strategy' : ['mean', 'median', 'most_frequent'],
    'randomforestclassifier__n_estimators' : [10, 50, 100],
    'randomforestclassifier__criterion' : ['gini', 'entropy'],
    'randomforestclassifier__max_depth' : [None, 25, 50, 100],
    'randomforestclassifier__min_samples_leaf' : [1, 7, 15, 0.1, 0.05],
    'randomforestclassifier__max_features' : [None, 'auto', 0.5, 0.75, 0.9],
}

search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_distributions, n_iter=50, scoring=None, cv=5, random_state=13, n_jobs=-1, verbose=10)

In [24]:
X_train_transformed = transformations.fit_transform(X_train)
X_test_transformed = transformations.transform(X_test)

search.fit(X_train_transformed, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  7.7min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 29.3min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 35.9min
[Parallel(n_jobs=-1)]: Done 

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,
                                                            missing_values=nan,
                                                            strategy='mean',
                                                            verbose=0)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(bootstrap=True,
                                                                     ccp_alpha=0.0,
                                                                     class_weight=None,
                                                        

In [25]:
print(search.best_params_)
print(search.best_score_)

{'simpleimputer__strategy': 'median', 'randomforestclassifier__n_estimators': 50, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_depth': None, 'randomforestclassifier__criterion': 'entropy'}
0.8103703703703703


## Submit your predictions to our Kaggle competition. 

(Go to our Kaggle InClass competition webpage. Use the blue Submit Predictions button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)

In [27]:
y_pred = search.best_estimator_.predict(X_test_transformed)

submission = sample_submission.copy()
submission[target] = y_pred

filename = 'jcs-kaggle-waterpumps-submission-04.csv'

submission.to_csv(filename, index=False)

submission

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14353,39307,non functional
14354,18990,functional
14355,28749,functional
14356,33492,functional


In [0]:
from google.colab import files
files.download(filename)